In [18]:
%cd ~
import pyfits,glob,time,scipy,sys
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from scipy.optimize import fmin_powell

topdir='/Users/cslage/Research/LSST/code/'
thedir=topdir+'galsim-git/GalSim/examples/brighter-fatter/output/bf_2/'
forwarddir = topdir+'galsim-git/GalSim/examples/brighter-fatter/forward_model_varying_i'
%cd $forwarddir
import  forward
%cd $thedir

/Users/cslage
/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/forward_model_varying_i
/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2


In [19]:
zfilelist=sort(glob.glob(thedir+'bf_?.fits'))
zfilelist

array([ '/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_1.fits',
       '/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_2.fits',
       '/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_3.fits',
       '/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_4.fits',
       '/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_5.fits'], 
      dtype='|S97')

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [8]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 0
    xcoomax = 800
    ycoomin = 0
    ycoomax = 800
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    img=pyfits.getdata(fitsfilename,0) 
    catname=fitsfilename[:-5]+'_SEX.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = img[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xcoord - xint - 1.0
            spotlist.yoffset[n] = ycoord - yint - 1.0     
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist



def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom

def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [20]:
nx = 9
ny = 9
numspots = 25
results = []
spotlists=[]
segmentnumber = 0

for fitsfilename in zfilelist:
    print fitsfilename
    param0 = [1.0, 1.0]
    
    spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny,0.2,2.0)
    print "nstamps = %d"%spotlist.nstamps

    spotlists.append(spotlist)
    args = ()#(spotlist)
    Result = fmin_powell(FOM, param0, args)
    print fitsfilename
    print Result
    
    results.append(Result)


/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_1.fits
nstamps = 25
Optimization terminated successfully.
         Current function value: 4432040.584486
         Iterations: 2
         Function evaluations: 52
/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_1.fits
[ 1.00577779  1.00700669]
/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_2.fits
nstamps = 25
Optimization terminated successfully.
         Current function value: 12390546.648285
         Iterations: 2
         Function evaluations: 53
/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_2.fits
[ 1.0102837   1.01299661]
/Users/cslage/Research/LSST/code/galsim-git/GalSim/examples/brighter-fatter/output/bf_2/bf_3.fits
nstamps = 25
Optimization terminated successfully.
         Current function value: 32178518.500961
         Iterations: 2
         Function eval

In [21]:
from scipy import stats
figure()
title("Brighter-Fatter Effect on GalSim Generated Spots", fontsize=12)
imaxs = []
sigmaxs = []
sigmays = []
gain = 1.0
for i,result in enumerate(results):
    imax = spotlists[i].imax.mean()
    ADU_correction = Area(-0.5,0.5,-0.5,0.5,result[0],result[1],1.0)
    imaxs.append(imax * ADU_correction * gain)
    sigmaxs.append(result[0])
    sigmays.append(result[1])
    #if result[2] < 1.06:
    #    print "I = %d, Imax = %.1f, sigmax = %.3f, sigmay = %.3f"%(i,result[0],result[1],result[2])
scatter(imaxs, sigmaxs, color = 'green', lw = 2, label = 'Sigma-x')
scatter(imaxs, sigmays, color = 'red', lw = 2, label = 'Sigma-y')

slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs,sigmaxs)

xplot=linspace(-5000.0,150000.0,100)
yplot = slope * xplot + intercept
plot(xplot, yplot, color='blue', lw = 2, ls = '--')
tslope = slope/intercept * 100.0 * 50000.0
text(5000,1.07,"X Slope = %.2f percent per 50,000 electrons"%tslope, fontsize=12)

slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs,sigmays)

xplot=linspace(-5000.0,150000.0,100)
yplot = slope * xplot + intercept
plot(xplot, yplot, color='black', lw = 2, ls = '--')
tslope = slope/intercept * 100.0 * 50000.0
text(5000,1.06,"Y Slope = %.2f percent per 50,000 electrons"%tslope, fontsize=12)

ylim(0.98,1.08)
xlim(0,160000)
xlabel('Central Peak(electrons)')
ylabel('Sigma (Pixels)')
legend(loc= 'lower right')
savefig("Forward_Model_Spots.png")

In [22]:
figure()
testn = 1
spot = 7
spotlist = spotlists[testn]
print spotlist.nstamps
Result = results[testn]
Imax = spotlist.imax[spot]
sigmax = Result[0]
sigmay = Result[1]
seqno = 0#int(zfilelist[testn].split("_")[3])
suptitle=("Sequence # %d, Spot # %d"%(seqno,spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)

subplots_adjust(wspace = 1.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot],interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot], lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
legend(loc = (-4.0,0.8))
savefig("Typical_Fit.png")


25
